#Install libraries

In [3]:
#for sentence based embeddings -- transformers give token embeddings, we need sentence embeddings
!pip install -qq sentence_transformers
#for working with llms
!pip install -qq -U langchain==0.1.0
#for pdf loading
!pip install -qq -U pypdf
#the vector store
!pip install -qq -U faiss-gpu
#for creating embeddings
!pip install -qq -U InstructorEmbedding
#for the llms
!pip install -qq -U transformers
#for quantized llms
!pip install -qq auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
#get drive access
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#load documents
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader
#change this path to use your own pdfs directory, all pdfs are within the same directory
pdf_directory = '/content/drive/MyDrive/HP books/'
#the pattern of files
pdf_pattern = './*.pdf'
loader = DirectoryLoader(
                         pdf_directory,
                         glob=pdf_pattern,
                         loader_cls=PyPDFLoader
)
documents = loader.load()

In [6]:
#split documents
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
                                          chunk_size=750,
                                          chunk_overlap=100
)
#split documents
documents = splitter.split_documents(documents)

In [7]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
#create embeddings
embeddings = HuggingFaceEmbeddings(
                                   model_kwargs={'device':'cuda'}
)
vectordb = FAISS.from_documents(
                                documents=documents,
                                embedding=embeddings
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
import transformers
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM
#model repo for loading
model_repo = '4bit/Llama-2-7b-Chat-GPTQ'
model_basename = 'gptq_model-4bit-128g'
#quantized model is loaded
model = AutoGPTQForCausalLM.from_quantized(
    model_repo,
    model_basename = model_basename,
    use_safetensors = True,
    trust_remote_code = True,
    device = 'cuda:0',
    use_triton = False,
    quantize_config = None
)
tokenizer = AutoTokenizer.from_pretrained(model_repo, use_fast = True)

config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

quantize_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

gptq_model-4bit-128g.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

INFO - The layer lm_head is not quantized.
INFO:auto_gptq.modeling._base:The layer lm_head is not quantized.


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [9]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
pipe = pipeline(
                task='text-generation',
                model=model,
                tokenizer=tokenizer,
                pad_token_id=tokenizer.eos_token_id,
                max_length=2048,
                temperature=0.5,
                do_sample=True,
                top_p=0.95,
                repetition_penalty=1.15
)
l_model = HuggingFacePipeline(pipeline = pipe)

The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MusicgenMelodyForCausalLM', 'MvpForCausalLM', 'OpenLlam

In [17]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
template = '''You are an expert on harry potter who is very helpful and factual.
Answer the question below based on the context given.
VERY IMPORTANT: "Do not make up anything", if you do not know, or are unsure, say that you do not know.
context:{context} question:{question} answer:'''# <$$$answer-begins-here$$$>:'''
prompt=PromptTemplate(template=template,input_variables=['context','question'])
retriever = vectordb.as_retriever(search_kwargs={'k':5,'search_type':'similarity'})
qa = RetrievalQA.from_chain_type(llm=l_model,chain_type='stuff',retriever=retriever,
                                 chain_type_kwargs={'prompt':prompt},return_source_documents=True,verbose=False)

In [18]:
a = qa.invoke('is harry a popular guy?')
print(a)

{'query': 'is harry a popular guy?', 'result': " Harry Potter is extremely well known among wizards and non magical people alike. He is considered a hero by many due to his role in defeating Lord Voldemort, also known as Tom Riddle. He has received numerous awards and recognition for his bravery and selflessness during this time. However, it's important to note that Harry himself is not particularly popular among his peers at Hogwarts School of Witchcraft and Wizardry, where he is in his seventh year. He has some friends such as Ron Weasley and Hermione Granger, but he is generally seen as being a bit awkward and unpopular among his classmates.", 'source_documents': [Document(page_content='he gets people fawning over him all the time. . . . Where d’you \nreckon they’re going to sleep? We  could offer him a space in our \ndormitory, Harry . . . I wouldn’t mind giving him my bed, I could \nkip on a camp bed.”', metadata={'source': '/content/drive/MyDrive/HP books/Harry Potter - Book 4 - 

In [ ]:
import langchain
print(langchain.__version__)

In [13]:
import textwrap
def prettify(query):
    response = qa.invoke(query)
    text = response['result']#.split('<$$$answer-begins-here$$$>:')[1]
    lines = text.split('\n')
    lines_wrap = [textwrap.fill(line,width=700) for line in lines]
    wrapped_text = '\n'.join(lines_wrap)
    sources = ' \n'.join([source.metadata['source'].split('/')[-1][:-4] + ' - page' + str(source.metadata['page']) for source in response['source_documents']])
    wrapped_text = wrapped_text + '\n'+ sources
    return wrapped_text

In [16]:
print(prettify("who is harry potter"))

:Harry James Potter (born 31 July 1980) is a British actor, voice actor, and film producer best known for his title role as Harry Potter in the Harry Potter film series.
Harry Potter - Book 6 - The Half-Blood Prince - page154 
Harry Potter - Book 2 - The Chamber of Secrets - page9 
Harry Potter - Book 6 - The Half-Blood Prince - page410 
Harry Potter - Book 4 - The Goblet of Fire - page14 
Harry Potter - Book 4 - The Goblet of Fire - page4
